This notebook includes all the code to preprocess datasets for the experiments in the paper. There will be two main parts: the first half is for row pairs preparation, while the second half is for attribute pairs preparation.

# Row Pairs Preparation

## Training & Validation Set

The magellan datasets and wdc dataset need different preparation steps. We will first prepare the magellan datasets.

In [22]:
import os
import pandas as pd
from autogluon.tabular import TabularPredictor

/mnt/scratch/mhussein/miniconda3/envs/anymatch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Magellan Datasets

In [8]:
magellan_dirs = {
    'abt': 'raw/abt_buy', 'amgo': 'raw/amazon_google',
    'beer': 'raw/beer', 'dbac': 'raw/dblp_acm',
    'dbgo': 'raw/dblp_scholar', 'foza': 'raw/fodors_zagat',
    'itam': 'raw/itunes_amazon', 'waam': 'raw/walmart_amazon',
}
magellan_dirs = {
    'music': 'raw/musicbrainz20k',
    'dbgo': 'raw/dblp_scholar', 
}

magellan_rename_columns = {
    'abt': ['id', 'name', 'description', 'price'], 'amgo': ['id', 'name', 'manufacturer', 'price'],
    'beer': ['id', 'name', 'factory', 'style', 'ABV'], 'dbac': ['id', 'title', 'authors', 'venue', 'year'],
    'dbgo': ['id', 'title', 'authors', 'venue', 'year'],
    'music': ['id', 'title', 'authors', 'venue', 'year'], 
    'foza': ['id', 'name', 'address', 'city', 'phone', 'type', 'class'],
    'itam': ['id', 'name', 'artist', 'album', 'genre', 'price', 'copyright', 'time', 'released'],
    'waam': ['id', 'name', 'category', 'brand', 'modelno', 'price'],
}

magellan_rename_columns = {
    'dbgo': ['id', 'title', 'authors', 'venue', 'year'],
    'music': ['id','cluster_id','source','number', 'title', 'length', 'artist', 'album', 'year','language'], 
}
magellan_drop_columns = {
    'abt': ['description'], 'amgo': ['manufacturer'], 'beer': [], 'dbac': [], 'dbgo': [], 'foza': [], 'itam': [],
    'waam': ['category', 'brand'],
}
magellan_drop_columns = {
    'dbgo': ['venue','year'], 
    'music': ['cluster_id', 'source','number','length','language'],
}

In [3]:
def merge_with_id(tableA, tableB, id_pairs):
    left_merged = pd.merge(tableA, id_pairs, left_on='id', right_on='ltable_id')
    left_right_merged = pd.merge(left_merged, tableB, left_on='rtable_id', right_on='id', suffixes=('_l', '_r'))
    left_right_merged.drop(columns=['ltable_id', 'rtable_id', 'id_l', 'id_r'], inplace=True)
    return left_right_merged

In [4]:
def prepare_magellan_row_pairs(dirs: dict, rename_columns: dict, drop_columns: dict):
    for d_name in dirs:
        tableA = pd.read_csv(os.path.join(dirs[d_name], 'tableA.csv'))
        tableB = pd.read_csv(os.path.join(dirs[d_name], 'tableB.csv'))
        tableA.columns = rename_columns[d_name]
        tableB.columns = rename_columns[d_name]
        tableA.drop(columns=drop_columns[d_name], inplace=True)
        tableB.drop(columns=drop_columns[d_name], inplace=True)

        train_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'train.csv'))
        valid_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'valid.csv'))
        test_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'test.csv'))
        train_df = merge_with_id(tableA, tableB, train_id_pairs)
        valid_df = merge_with_id(tableA, tableB, valid_id_pairs)
        test_df = merge_with_id(tableA, tableB, test_id_pairs)

        if not os.path.exists(f'prepared/{d_name}'):
            os.makedirs(f'prepared/{d_name}')
        train_df.to_csv(f'prepared/{d_name}/train.csv', index=False)
        valid_df.to_csv(f'prepared/{d_name}/valid.csv', index=False)
        test_df.to_csv(f'prepared/{d_name}/test.csv', index=False)

In [13]:
prepare_magellan_row_pairs(magellan_dirs, magellan_rename_columns, magellan_drop_columns)

### WDC Datasets

In [12]:
def prepare_wdc_row_pairs(dir: str,product='computers'):
    used_columns = ['title_left', 'price_left', 'priceCurrency_left', 'label', 'title_right', 'price_right', 'priceCurrency_right']
    used_columns = ['title_left','description_left','brand_left', 'label',
                    # 'price_left',
                'title_right','description_right','brand_right',
                # 'price_right',
                ]
    train_df = pd.read_csv(os.path.join(dir, 'train.csv'))[used_columns]
    valid_df = pd.read_csv(os.path.join(dir, 'valid.csv'))[used_columns]
    test_df = pd.read_csv(os.path.join(dir, 'test.csv'))[used_columns]

    # merge_price_currency = lambda x, y: str(y) + str(x) if pd.notna(x) and pd.notna(y) else None
    # train_df['price_left'] = train_df.apply(lambda x: merge_price_currency(x['price_left'], x['priceCurrency_left']), axis=1)
    # train_df['price_right'] = train_df.apply(lambda x: merge_price_currency(x['price_right'], x['priceCurrency_right']), axis=1)
    # train_df.drop(columns=['priceCurrency_left', 'priceCurrency_right'], inplace=True)
    # train_df.columns = ['title_l', 'price_l', 'label', 'title_r', 'price_r']
    train_df.columns = ['title_l', 'description_l', 'brand_l','label', 'title_r', 'description_r','brand_r']

    # valid_df['price_left'] = valid_df.apply(lambda x: str(x['price_left'])+ str(x['priceCurrency_left']), axis=1)
    # valid_df['price_right'] = valid_df.apply(lambda x: str(x['price_right'])+ str(x['priceCurrency_right']), axis=1)
    # valid_df.drop(columns=['priceCurrency_left', 'priceCurrency_right'], inplace=True)
    # valid_df.columns = ['title_l', 'price_l', 'label', 'title_r', 'price_r']
    valid_df.columns =  ['title_l', 'description_l', 'brand_l','label', 'title_r', 'description_r','brand_r']

    # test_df['price_left'] = test_df.apply(lambda x: str(x['price_left'])+ str(x['priceCurrency_left']), axis=1)
    # test_df['price_right'] = test_df.apply(lambda x: str(x['price_right'])+ str(x['priceCurrency_right']), axis=1)
    # test_df.drop(columns=['priceCurrency_left', 'priceCurrency_right'], inplace=True)
    # test_df.columns = ['title_l', 'price_l', 'label', 'title_r', 'price_r']
    test_df.columns = ['title_l', 'description_l', 'brand_l','label', 'title_r', 'description_r','brand_r']
    if not os.path.exists(f'prepared/wdc-{product}'):
        os.makedirs(f'prepared/wdc-{product}')
    train_df.to_csv(f'prepared/wdc-{product}/train.csv', index=False)
    valid_df.to_csv(f'prepared/wdc-{product}/valid.csv', index=False)
    test_df.to_csv(f'prepared/wdc-{product}/test.csv', index=False)

In [24]:
import pandas as pd
import os
products =['computers','watches','shoes','cameras']
for product in products:
    prepare_wdc_row_pairs(f'raw/wdc-{product}',product=product)

In [5]:
# #test wdc created correctly
# product='watches'
# df = pd.read_csv(f'prepared/wdc-{product}/valid.csv')
# print(df.head())

                                             title_l  \
0  seiko men s prospex diver padi special edition...   
1  billet egypte 25 piastres mosqu e al sayida ai...   
2  alpina pilot quartz chronograph watch al 372lb...   
3   darkony swatch malaysia xlite 41 45 mm yys4006ag   
4  carl f bucherer t graph watch 00 10615 03 33 2...   

                                       description_l brand_l  label  \
0  this gents seiko prospex diver padi special ed...     NaN      0   
1  mosqu e al sayida aisha sign 22date mission 20...     NaN      0   
2  p an intricate watch will always deliver sophi...     NaN      0   
3                                                NaN     NaN      1   
4  p at costello jewelry company in naperville il...     NaN      1   

                                             title_r  \
0  zegarek damski le locle tissot t41 1 183 33 nu...   
1  daniel wellington classic lady black bristol r...   
2  alpina startimer pilot automatic watch al 725b...   
3  darkony s

## Test Set

### Magellan Datasets
The previous steps will generate a test set for each magellan dataset, while some of them will be overwritten by the following code.

In [ ]:
# abt_buy
used_columns = ['name_left', 'price_left', 'label', 'name_right', 'price_right']
renamed_columns = ['name_l', 'price_l', 'label', 'name_r', 'price_r']
abt_df = pd.read_pickle('raw/abt_buy/test.pkl.gz')[used_columns]
abt_df.columns = renamed_columns
abt_df.to_csv('prepared/abt/test.csv', index=False)

In [6]:
# amgo
test_magellan_used_columns = {
    'abt': ['name_left', 'price_left', 'label', 'name_right', 'price_right'],
    'amgo': ['title_left', 'price_left', 'label', 'title_right', 'price_right'],
    'dbac': ['title_left', 'authors_left', 'venue_left', 'year_left', 'label', 'title_right', 'authors_right', 'venue_right', 'year_right'],
    'dbgo': ['title_left', 'authors_left', 'venue_left', 'year_left', 'label', 'title_right', 'authors_right', 'venue_right', 'year_right'],
    'waam': ['title_left', 'modelno_left', 'price_left', 'label', 'title_right', 'modelno_right', 'price_right']
}
test_magellan_used_columns = {
    'dbgo': ['title_left', 'authors_left', 'label', 'title_right', 'authors_right', ],
    'music': ['title_left', 'artist_left', 'album_left','year_left', 'label', 'title_right', 'artist_right', 'album_right','year_right']
}

test_magellan_rename_columns = {
    'abt': ['name_l', 'price_l', 'label', 'name_r', 'price_r'],
    'amgo': ['name_l', 'price_l', 'label', 'name_r', 'price_r'],
    'dbac': ['title_l', 'authors_l', 'venue_l', 'year_l', 'label', 'title_r', 'authors_r', 'venue_r', 'year_r'],
    'dbgo': ['title_l', 'authors_l', 'venue_l', 'year_l', 'label', 'title_r', 'authors_r', 'venue_r', 'year_r'],
    'waam': ['name_l', 'modelno_l', 'price_l', 'label', 'name_r', 'modelno_r', 'price_r']
}
test_magellan_rename_columns = {
    'dbgo': ['title_l', 'authors_l', 'label', 'title_r', 'authors_r', ],
    'music': ['title_l', 'artist_l', 'album_l', 'year_l','label', 'title_r', 'artist_r', 'album_r','year_r']
}

def prepare_test_magellan_row_pairs(dirs: dict, used_columns: dict, rename_columns: dict):
    dirs = {key: dirs[key] for key in used_columns.keys() if key in dirs}
    for d_name in dirs:
        d_used_columns = used_columns[d_name]
        d_rename_columns = rename_columns[d_name]
        df = pd.read_csv(f'{dirs[d_name]}/test.csv')[d_used_columns]
        df.columns = d_rename_columns
        df.to_csv(f'prepared/{d_name}/test.csv', index=False)

In [9]:
prepare_test_magellan_row_pairs(magellan_dirs, test_magellan_used_columns, test_magellan_rename_columns)

KeyError: "['title_left', 'authors_left', 'title_right', 'authors_right'] not in index"

### WDC Datasets

In [ ]:
def prepare_test_wdc_row_pairs(dir: str):
    used_columns = ['title_left', 'price_left', 'priceCurrency_left', 'label', 'title_right', 'price_right', 'priceCurrency_right']
    test_df = pd.read_pickle(os.path.join(dir, 'test.pkl.gz'))[used_columns]

    merge_price_currency = lambda x, y: str(y) + str(x) if pd.notna(x) and pd.notna(y) else None
    test_df['price_left'] = test_df.apply(lambda x: merge_price_currency(x['price_left'], x['priceCurrency_left']), axis=1)
    test_df['price_right'] = test_df.apply(lambda x: merge_price_currency(x['price_right'], x['priceCurrency_right']), axis=1)
    test_df.drop(columns=['priceCurrency_left', 'priceCurrency_right', 'price_left', 'price_right'], inplace=True)
    # test_df.columns = ['title_l', 'price_l', 'label', 'title_r', 'price_r']
    test_df.columns = ['title_l', 'label', 'title_r'] # to align with the MatchGPT paper

    test_df.to_csv(f'prepared/wdc/test.csv', index=False)

In [ ]:
# prepare_test_wdc_row_pairs('raw/wdc')

# Attribute Pairs Preparation

In [15]:
dataset_names = ['abt', 'amgo', 'beer', 'dbac', 'dbgo', 'foza', 'itam', 'waam', 'wdc']
dataset_names = [f'wdc-{product}' for product in products]
dataset_names.extend(['dbgo', 'music',])

In [16]:
def nan_check(value):
    null_strings = [None, 'nan', 'NaN', 'NAN', 'null', 'NULL', 'Null', 'None', 'none', 'NONE', '', '-', '--', '---']
    if pd.isna(value) or pd.isnull(value) or value in null_strings:
        return 1
    else:
        return 0

def numerical_check(value):
    if isinstance(value, int) or isinstance(value, float):
        return 1

def string_identical_check(left_value, right_value, row_label):
    if left_value == right_value or left_value in right_value or right_value in left_value:
        return 1
    else:
        if row_label == 1:
            return 1
        else:
            return 0

def numerical_identical_check(left_value, right_value, row_label):
    if left_value == right_value:
        return 1
    else:
        return 0

def identical_check(left_value, right_value, row_label):
    if nan_check(left_value) and not nan_check(right_value):
        return 0
    elif not nan_check(left_value) and nan_check(right_value):
        return 0
    elif nan_check(left_value) and nan_check(right_value):
        return 1
    elif numerical_check(left_value) and numerical_check(right_value):
        return numerical_identical_check(left_value, right_value, row_label)
    else:
        left_value = str(left_value).lower()
        right_value = str(right_value).lower()
        return string_identical_check(left_value, right_value, row_label)

In [17]:
def row2attribute_pairs(row):
    attr_pairs = []
    all_columns = row.index
    left_columns = [col for col in all_columns if col.endswith('_l')]
    right_columns = [col for col in all_columns if col.endswith('_r')]
    row_label = row['label']
    for i in range(len(left_columns)):
        left_value = row[left_columns[i]]
        right_value = row[right_columns[i]]
        attr_pair = [left_value, right_value, identical_check(left_value, right_value, row_label), left_columns[i][:-2]]
        attr_pairs.append(attr_pair)
    return attr_pairs

In [18]:
def prepare_all_attribute_pairs(names: list):
    for name in names:
        train_row_pairs = pd.read_csv(f'prepared/{name}/train.csv')
        valid_row_pairs = pd.read_csv(f'prepared/{name}/valid.csv')
        test_row_pairs = pd.read_csv(f'prepared/{name}/test.csv')
        train_attr_pairs = []
        valid_attr_pairs = []
        test_attr_pairs = []

        train_row_pairs.apply(lambda row: train_attr_pairs.extend(row2attribute_pairs(row)), axis=1)
        valid_row_pairs.apply(lambda row: valid_attr_pairs.extend(row2attribute_pairs(row)), axis=1)
        test_row_pairs.apply(lambda row: test_attr_pairs.extend(row2attribute_pairs(row)), axis=1)

        train_attr_pairs_df = pd.DataFrame(train_attr_pairs, columns=['left_value', 'right_value', 'label', 'attribute'])
        val_attr_pairs_df = pd.DataFrame(valid_attr_pairs, columns=['left_value', 'right_value', 'label', 'attribute'])
        test_attr_pairs_df = pd.DataFrame(test_attr_pairs, columns=['left_value', 'right_value', 'label', 'attribute'])
        train_attr_pairs_df.drop_duplicates(inplace=True)
        val_attr_pairs_df.drop_duplicates(inplace=True)
        test_attr_pairs_df.drop_duplicates(inplace=True)

        train_attr_pairs_df.to_csv(f'prepared/{name}/attr_train.csv', index=False)
        val_attr_pairs_df.to_csv(f'prepared/{name}/attr_valid.csv', index=False)
        test_attr_pairs_df.to_csv(f'prepared/{name}/attr_test.csv', index=False)

In [25]:
prepare_all_attribute_pairs(dataset_names)

# AutoML Predictions

In [27]:
def prepare_automl_predictions():
    dataset_names = ['abt', 'amgo', 'beer', 'dbac', 'dbgo', 'foza', 'itam', 'waam', 'wdc']
    dataset_names = [f'wdc-{product}' for product in products]
    dataset_names.extend(['dbgo', 'music',])
    dataset_names= [dataset_names[3]]
    for name in dataset_names:
        train_df = pd.read_csv(f'prepared/{name}/train.csv')
        valid_df = pd.read_csv(f'prepared/{name}/valid.csv')

        predictor = TabularPredictor(label='label').fit(train_data=train_df, tuning_data=valid_df, verbosity=-1)
        train_preds = predictor.predict(train_df)
        train_preds_proba = predictor.predict_proba(train_df)
        valid_preds = predictor.predict(valid_df)
        valid_preds_proba = predictor.predict_proba(valid_df)
        train_preds_df = pd.DataFrame({'prediction': train_preds, 'proba_0': train_preds_proba[0], 'proba_1': train_preds_proba[1]})
        valid_preds_df = pd.DataFrame({'prediction': valid_preds, 'proba_0': valid_preds_proba[0], 'proba_1': valid_preds_proba[1]})

        if not os.path.exists(f'automl/{name}'):
            os.makedirs(f'automl/{name}')
        train_preds_df.to_csv(f'automl/{name}/train_preds.csv', index=False)
        valid_preds_df.to_csv(f'automl/{name}/valid_preds.csv', index=False)

In [28]:
prepare_automl_predictions()

No path specified. Models will be saved in: "AutogluonModels/ag-20250204_140023"
